# Football Data from Transfermarkt
- Kaggle Data URL

    - https://www.kaggle.com/datasets/davidcariboo/player-scores

## 데이터 설명
- Transfermarkt 웹사이트에서 스크레이핑한 각종 축구 데이터 (매주 한번 파일 업데이트)
    - 주요 대회 시즌별 60,000개 이상의 경기
    - 모든 대회의 400개 이상의 클럽들
    - 30,000 이상의 선수들
    - 400,000개 이상의 선수 가치 기록
    - 1,200,000개 이상의 선수 출전 기록

- `csv` 파일 설명
    - `apperances` : 선수 출장 기록
    - `club_games` : 클럽별 경기 홈팀, 어웨이팀 정보
    - `clubs` : 리그별 속해 있는 클럽
    - `competitions` : 대회 정보
    - `game_events` : 경기별 이벤트 정보 (카드, 득점, 어시스트 등)
    - `game_lineups` : 경기별 선수의 선발, 교체 명단 등재 여부
    - `games` : 경기에 대한 정보 (시즌, 라운드, 홈팀, 어웨이팀, 순위)
    - `player_valuations` : Transfermarkt 웹사이트에서 매긴 선수의 가치
    - `players` : 선수에 대한 세부 정보

- 데이터베이스 스키마
    <img src="https://raw.githubusercontent.com/dcaribou/transfermarkt-datasets/master/resources/diagram.svg?sanitize=true" width="1700">

In [25]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from datetime import datetime
from datetime import timedelta
import missingno as msno

In [26]:
# 데이터 경로 지정
# C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data
apps_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/appearances.csv"
club_games_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/club_games.csv"
clubs_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/clubs.csv"
competitions_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/competitions.csv"
game_events_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/game_events.csv"
game_lineups_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/game_lineups.csv"
games_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/games.csv"
player_valuations_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/player_valuations.csv"
players_path = "C:/Users/aryij/Documents/DataStudy/football-data-from-transfermarkt-data/players.csv"

# 데이터셋 불러오기
apps_df = pd.read_csv(apps_path)
club_games_df = pd.read_csv(club_games_path)
clubs_df = pd.read_csv(clubs_path)
competitions_df = pd.read_csv(competitions_path)
game_events_df = pd.read_csv(game_events_path)
game_lineups_df = pd.read_csv(game_lineups_path)
games_df = pd.read_csv(games_path)
player_valuations_df = pd.read_csv(player_valuations_path)
players_df = pd.read_csv(players_path)

In [27]:
apps_df.head(3)

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
0,2231978_38004,2231978,38004,853,235,2012-07-03,Aurélien Joachim,CLQ,0,0,2,0,90
1,2233748_79232,2233748,79232,8841,2698,2012-07-05,Ruslan Abyshov,ELQ,0,0,0,0,90
2,2234413_42792,2234413,42792,6251,465,2012-07-05,Sander Puri,ELQ,0,0,0,0,45


In [28]:
club_games_df.head(3)

,game_id,club_id,own_goals,own_position,own_manager_name,opponent_id,opponent_goals,opponent_position,opponent_manager_name,hosting,is_win
0,2320450,1468,0,NaN,Holger Bachthaler,24,2,NaN,Armin Veh,Home,0
1,2320460,1,3,NaN,Jürgen Luginger,86,1,NaN,Robin Dutt,Home,1
2,2320472,2036,4,NaN,Frank Schmidt,72,5,NaN,Alexander Schmidt,Home,0


In [29]:
clubs_df.tail(3)

,club_id,club_code,name,domestic_competition_id,total_market_value,squad_size,average_age,foreigners_number,foreigners_percentage,national_team_players,stadium_name,stadium_seats,net_transfer_record,coach_name,last_season,filename,url
423,800,atalanta-bergamo,Atalanta Bergamasca Calcio S.p.a.,IT1,NaN,25,26.5,17,68.0,16,Gewiss Stadium,21747,+€73.82m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/atalanta-berga...
424,979,moreirense-fc,Moreirense Futebol Clube,PO1,NaN,29,25.5,17,58.6,3,Estádio C. J. de Almeida Freitas,6153,€-2.05m,NaN,2023,../data/raw/transfermarkt-scraper/2023/clubs.j...,https://www.transfermarkt.co.uk/moreirense-fc/...
425,984,west-bromwich-albion,West Bromwich Albion,GB1,NaN,25,27.8,8,32.0,4,The Hawthorns,26850,€-1.35m,NaN,2020,../data/raw/transfermarkt-scraper/2020/clubs.j...,https://www.transfermarkt.co.uk/west-bromwich-...


In [30]:
competitions_df.head(3)

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url
0,CIT,italy-cup,italy-cup,domestic_cup,domestic_cup,75,Italy,IT1,europa,https://www.transfermarkt.co.uk/italy-cup/star...
1,NLSC,johan-cruijff-schaal,johan-cruijff-schaal,domestic_super_cup,other,122,Netherlands,NL1,europa,https://www.transfermarkt.co.uk/johan-cruijff-...
2,GRP,kypello-elladas,kypello-elladas,domestic_cup,domestic_cup,56,Greece,GR1,europa,https://www.transfermarkt.co.uk/kypello-ellada...


In [31]:
game_events_df.head(3)

,game_event_id,date,game_id,minute,type,club_id,player_id,description,player_in_id,player_assist_id
0,2f41da30c471492e7d4a984951671677,2012-08-05,2211607,77,Cards,610,4425,"1. Yellow card , Mass confrontation",NaN,NaN
1,a72f7186d132775f234d3e2f7bc0ed5b,2012-08-05,2211607,77,Cards,383,33210,"1. Yellow card , Mass confrontation",NaN,NaN
2,b2d721eaed4692a5c59a92323689ef18,2012-08-05,2211607,3,Goals,383,36500,", Header, 1. Tournament Goal Assist: , Corner,...",NaN,56416.0


In [32]:
game_lineups_df.head(3)

,game_lineups_id,game_id,club_id,type,number,player_id,player_name,team_captain,position
0,b2dbe01c3656b06c8e23e9de714e26bb,2317258,610,substitutes,5,1443,Christian Poulsen,0,Defensive Midfield
1,b50a3ec6d52fd1490aab42042ac4f738,2317258,610,starting_lineup,4,5017,Niklas Moisander,0,Centre-Back
2,7d890e6d0ff8af84b065839966a0ec81,2317258,1090,substitutes,11,9602,Maarten Martens,0,Left Winger


In [33]:
games_df.head(3)

,game_id,competition_id,season,round,date,home_club_id,away_club_id,home_club_goals,away_club_goals,home_club_position,...,stadium,attendance,referee,url,home_club_formation,away_club_formation,home_club_name,away_club_name,aggregate,competition_type
0,2321044,L1,2013,2. Matchday,2013-08-18,16,23,2,1,1.0,...,SIGNAL IDUNA PARK,80200.0,Peter Sippel,https://www.transfermarkt.co.uk/borussia-dortm...,4-2-3-1,4-3-2-1,Borussia Dortmund,Eintracht Braunschweig,2:1,domestic_league
1,2321060,L1,2013,3. Matchday,2013-08-25,23,24,0,2,18.0,...,EINTRACHT-Stadion,23325.0,Wolfgang Stark,https://www.transfermarkt.co.uk/eintracht-brau...,4-3-2-1,4-2-3-1,Eintracht Braunschweig,Eintracht Frankfurt Fußball AG,0:2,domestic_league
2,2321086,L1,2013,6. Matchday,2013-09-21,4,16,1,1,15.0,...,Max-Morlock-Stadion,50000.0,Knut Kircher,https://www.transfermarkt.co.uk/1-fc-nuremberg...,4-2-3-1,4-2-3-1,1.FC Nuremberg,Borussia Dortmund,1:1,domestic_league


In [34]:
player_valuations_df.head(3)

,player_id,date,market_value_in_eur,current_club_id,player_club_domestic_competition_id
0,405973,2000-01-20,150000,3057,BE1
1,342216,2001-07-20,100000,1241,SC1
2,3132,2003-12-09,400000,126,TR1


In [35]:
players_df.head(3)

,player_id,first_name,last_name,name,last_season,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,10,Miroslav,Klose,Miroslav Klose,2015,398,miroslav-klose,Poland,Opole,Germany,...,right,184.0,NaN,ASBW Sport Marketing,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,IT1,Società Sportiva Lazio S.p.A.,1000000.0,30000000.0
1,26,Roman,Weidenfeller,Roman Weidenfeller,2017,16,roman-weidenfeller,Germany,Diez,Germany,...,left,190.0,NaN,Neubauer 13 GmbH,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roman-weidenfe...,L1,Borussia Dortmund,750000.0,8000000.0
2,65,Dimitar,Berbatov,Dimitar Berbatov,2015,1091,dimitar-berbatov,Bulgaria,Blagoevgrad,Bulgaria,...,NaN,NaN,NaN,CSKA-AS-23 Ltd.,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/dimitar-berbat...,GR1,Panthessalonikios Athlitikos Omilos Konstantin...,1000000.0,34500000.0


### `.info()` 확인

In [36]:
# df.info() 확인

dfs = [apps_df, club_games_df, clubs_df, competitions_df, game_events_df, game_lineups_df, 
       games_df, player_valuations_df, players_df]

for df in dfs:
    print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1540638 entries, 0 to 1540637
Data columns (total 13 columns):
 #   Column                  Non-Null Count    Dtype 
---  ------                  --------------    ----- 
 0   appearance_id           1540638 non-null  object
 1   game_id                 1540638 non-null  int64 
 2   player_id               1540638 non-null  int64 
 3   player_club_id          1540638 non-null  int64 
 4   player_current_club_id  1540638 non-null  int64 
 5   date                    1540638 non-null  object
 6   player_name             1540637 non-null  object
 7   competition_id          1540638 non-null  object
 8   yellow_cards            1540638 non-null  int64 
 9   red_cards               1540638 non-null  int64 
 10  goals                   1540638 non-null  int64 
 11  assists                 1540638 non-null  int64 
 12  minutes_played          1540638 non-null  int64 
dtypes: int64(9), object(4)
memory usage: 152.8+ MB
None
<class 'pandas.core.

In [37]:
apps_copy = apps_df.copy()

### 사용 데이터셋
- 목표 : 선수의 시즌별 스탯 확인
    - `apperances` : 선수 출장 기록
    - `competitions` : 대회 정보
    - `players` : 선수에 대한 세부 정보
    
- 해볼 수 있는 것
    - 선수 스탯 예측
    - 선수 몸값 예측

In [38]:
apps_df.describe(include="all")

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,goals,assists,minutes_played
count,1540638,1.540638e+06,1.540638e+06,1.540638e+06,1.540638e+06,1540638,1540637,1540638,1.540638e+06,1.540638e+06,1.540638e+06,1.540638e+06,1.540638e+06
unique,1540638,NaN,NaN,NaN,NaN,3416,23472,43,NaN,NaN,NaN,NaN,NaN
top,2231978_38004,NaN,NaN,NaN,NaN,2020-10-04,Danilo,IT1,NaN,NaN,NaN,NaN,NaN
freq,1,NaN,NaN,NaN,NaN,1795,1062,127333,NaN,NaN,NaN,NaN,NaN
mean,NaN,3.045293e+06,1.855729e+05,2.967665e+03,3.642055e+03,NaN,NaN,NaN,1.486358e-01,3.806864e-03,9.627894e-02,7.582248e-02,6.956878e+01
std,NaN,5.774826e+05,1.704357e+05,7.534551e+03,9.289958e+03,NaN,NaN,NaN,3.670711e-01,6.158226e-02,3.316466e-01,2.862993e-01,2.979700e+01
min,NaN,2.211607e+06,1.000000e+01,1.000000e+00,-1.000000e+00,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
25%,NaN,2.545933e+06,5.528200e+04,2.810000e+02,3.360000e+02,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,5.100000e+01
50%,NaN,2.981997e+06,1.299910e+05,8.260000e+02,9.310000e+02,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+01
75%,NaN,3.497453e+06,2.700510e+05,2.441000e+03,2.687000e+03,NaN,NaN,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.000000e+01


In [39]:
competitions_df.describe(include="all")

,competition_id,competition_code,name,sub_type,type,country_id,country_name,domestic_league_code,confederation,url
count,43,43,43,43,43,43.000000,36,36,43,43
unique,43,42,42,11,4,NaN,14,14,1,43
top,CIT,premier-liga,premier-liga,first_tier,domestic_league,NaN,England,GB1,europa,https://www.transfermarkt.co.uk/italy-cup/star...
freq,1,2,2,14,14,NaN,4,4,43,1
mean,NaN,NaN,NaN,NaN,NaN,97.093023,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN,NaN,69.766896,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN,NaN,-1.000000,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN,NaN,39.500000,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN,NaN,122.000000,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN,NaN,157.000000,NaN,NaN,NaN,NaN


In [40]:
players_df.describe()

,player_id,last_season,current_club_id,height_in_cm,market_value_in_eur,highest_market_value_in_eur
count,3.049500e+04,30495.000000,30495.000000,28410.000000,2.935200e+04,2.935200e+04
mean,3.142658e+05,2018.790589,4416.199115,182.250827,1.574004e+06,3.635859e+06
std,2.531242e+05,3.651044,10149.194332,6.841391,6.154743e+06,9.463744e+06
min,1.000000e+01,2012.000000,3.000000,18.000000,1.000000e+04,1.000000e+04
25%,9.598000e+04,2016.000000,403.000000,178.000000,1.000000e+05,2.750000e+05
50%,2.590250e+05,2020.000000,1075.000000,182.000000,2.500000e+05,8.000000e+05
75%,4.704080e+05,2022.000000,3057.000000,187.000000,6.500000e+05,2.800000e+06
max,1.229924e+06,2023.000000,83678.000000,207.000000,1.800000e+08,2.000000e+08


In [41]:
players_df.describe(include="object")

,first_name,last_name,name,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,contract_expiration_date,agent_name,image_url,url,current_club_domestic_competition_id,current_club_name
count,28505,30495,30495,30495,27773,28294,29919,30450,30321,30495,28122,19041,15115,30495,30495,30495,30495
unique,6596,22453,29847,29810,184,8227,182,8931,13,5,3,100,2671,24940,30495,14,424
top,David,García,Paulinho,paulinho,France,London,Spain,1996-01-19,Centre-Back,Defender,right,2024-06-30 00:00:00,Wasserman,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/miroslav-klose...,TR1,Kilmarnock Football Club
freq,203,61,12,12,2185,434,1813,20,5357,9703,19733,5158,435,5556,1,3005,181


#### 사용 데이터셋 병합 함수 생성

In [42]:
# appearance_df, competitions_df, players_df merge 후 19-20 ~ 23-24 시즌 유럽 6대리그 데이터만 남김 (5시즌)
def app_comp_player_merge(apps_copy, competitions_df, players_df):
    # 데이터프레임의 date 컬럼 타입 datetime으로 변경
    apps_copy["date"] = pd.to_datetime(apps_copy["date"])
    
    # apps_copy와 competition_df competition_id를 기준으로 left join
    app_comp_df = pd.merge(apps_copy, competitions_df, how="left", on="competition_id")
    
    # 19-20 ~ 23-24 시즌 데이터만 남김 (5시즌)
    app_comp_df = app_comp_df[app_comp_df["date"]>="2019-06"]

    app_comp_player_df = pd.merge(app_comp_df, players_df, how="left", on="player_id")

#     cols = ['game_id', 'player_id', 'player_club_id',
#             'player_current_club_id', 'date', 'player_name', 'competition_id',
#             'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#             'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#             'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#             'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   
     
#     cols2 = ['player_id', 'player_club_id',
#              'player_current_club_id', 'date', 'player_name', 'competition_id',
#              'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#              'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#              'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#              'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   

    # 유럽 6대리그 (잉글랜드, 스페인, 이탈리아, 독일, 프랑스, 포르투갈 1부리그)만 필터링
    league_cond = ((app_comp_player_df.domestic_league_code=="GB1") | (app_comp_player_df.domestic_league_code=="ES1") | 
                   (app_comp_player_df.domestic_league_code=="IT1") | (app_comp_player_df.domestic_league_code=="L1") |
                   (app_comp_player_df.domestic_league_code=="FR1") | (app_comp_player_df.domestic_league_code=="PO1"))
#     app_comp_player_df = app_comp_player_df[cols2][league_cond].reset_index(drop=True)
    app_comp_player_df = app_comp_player_df[league_cond].reset_index(drop=True)

    # last_season 제대로 반영되지 않은 것들 발견하여, 새로 변경해준다
    # 6월을 넘지 않으면 직전 시즌 연도로 변경
    app_comp_player_df["last_season"] = app_comp_player_df["date"].apply(lambda x : x.year if x.month>=6 else x.year-1)

    # 수치형 변수 column 리스트
    group_cols = ["player_name", "last_season", "yellow_cards", "red_cards","goals", "assists", "minutes_played"]
    
    # groupby를 통해, 이름, 연도 기준으로 수치형 변수 합산 -> 시즌별 기록 파악하기 위함
    df_grouped = app_comp_player_df[group_cols].groupby(["player_name", "last_season"]).sum()
    df_grouped.reset_index(inplace=True)

#     # merge 하기 위한 기준인 "player_name", "last_season"은 남겨두고 group_cols와 일치하지 않는 컬럼 삭재
#     df_not_grouped = app_comp_player_df.loc[:, ~app_comp_player_df.columns.isin(group_cols[2:])]

#     # 위에서 만든 df_grouped, df_not_grouped merge
#     df_merged = pd.merge(df_grouped, df_not_grouped, how="left", on=["player_name", "last_season"])
    
    return df_grouped
#     return df_merged

In [43]:
app_comp_player_merge(apps_copy, competitions_df, players_df)


,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played
0,Aaron Boupendza,2019,0,0,0,0,90
1,Aaron Connolly,2019,0,0,2,1,878
2,Aaron Connolly,2020,0,0,3,2,1219
3,Aaron Connolly,2021,0,0,0,0,243
4,Aaron Connolly,2022,0,0,0,0,29
...,...,...,...,...,...,...,...
16615,İlkay Gündoğan,2019,7,0,3,2,1770
16616,İlkay Gündoğan,2020,1,0,14,4,2810
16617,İlkay Gündoğan,2021,1,0,10,4,2273
16618,İlkay Gündoğan,2022,1,0,8,5,2769


In [44]:
app_comp_player_merge(apps_copy, competitions_df, players_df)

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played
0,Aaron Boupendza,2019,0,0,0,0,90
1,Aaron Connolly,2019,0,0,2,1,878
2,Aaron Connolly,2020,0,0,3,2,1219
3,Aaron Connolly,2021,0,0,0,0,243
4,Aaron Connolly,2022,0,0,0,0,29
...,...,...,...,...,...,...,...
16615,İlkay Gündoğan,2019,7,0,3,2,1770
16616,İlkay Gündoğan,2020,1,0,14,4,2810
16617,İlkay Gündoğan,2021,1,0,10,4,2273
16618,İlkay Gündoğan,2022,1,0,8,5,2769


In [45]:
df_merged = app_comp_player_merge(apps_copy, competitions_df, players_df)
df_merged.head(3)

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played
0,Aaron Boupendza,2019,0,0,0,0,90
1,Aaron Connolly,2019,0,0,2,1,878
2,Aaron Connolly,2020,0,0,3,2,1219


In [46]:
df_merged.shape

(16620, 7)

In [47]:
df_merged

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played
0,Aaron Boupendza,2019,0,0,0,0,90
1,Aaron Connolly,2019,0,0,2,1,878
2,Aaron Connolly,2020,0,0,3,2,1219
3,Aaron Connolly,2021,0,0,0,0,243
4,Aaron Connolly,2022,0,0,0,0,29
...,...,...,...,...,...,...,...
16615,İlkay Gündoğan,2019,7,0,3,2,1770
16616,İlkay Gündoğan,2020,1,0,14,4,2810
16617,İlkay Gündoğan,2021,1,0,10,4,2273
16618,İlkay Gündoğan,2022,1,0,8,5,2769


In [48]:
df_merged.to_csv("data/df_merged.csv", index=False)

In [174]:
# appearance_df, competitions_df, players_df merge 후 19-20 ~ 23-24 시즌 유럽 6대리그 데이터만 남김 (5시즌)
def app_comp_player_merge2(apps_copy, competitions_df, players_df):
    # 데이터프레임의 date 컬럼 타입 datetime으로 변경
    apps_copy["date"] = pd.to_datetime(apps_copy["date"])
    
    # apps_copy와 competition_df competition_id를 기준으로 left join
    app_comp_df = pd.merge(apps_copy, competitions_df, how="left", on="competition_id")
    
    # 19-20 ~ 23-24 시즌 데이터만 남김 (5시즌)
    app_comp_df = app_comp_df[app_comp_df["date"]>="2019-06"]

    app_comp_player_df = pd.merge(app_comp_df, players_df, how="left", on="player_id")

#     cols = ['game_id', 'player_id', 'player_club_id',
#             'player_current_club_id', 'date', 'player_name', 'competition_id',
#             'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#             'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#             'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#             'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   
     
#     cols2 = ['player_id', 'player_club_id',
#              'player_current_club_id', 'date', 'player_name', 'competition_id',
#              'yellow_cards', 'red_cards', 'goals', 'assists', 'minutes_played',
#              'competition_code', 'country_id', 'country_name', 'domestic_league_code', 'last_season',
#              'country_of_citizenship', 'date_of_birth', 'sub_position', 'foot', 'height_in_cm', 
#              'contract_expiration_date', 'current_club_name', 'market_value_in_eur', 'highest_market_value_in_eur']   

    # 유럽 6대리그 (잉글랜드, 스페인, 이탈리아, 독일, 프랑스, 포르투갈 1부리그)만 필터링
    league_cond = ((app_comp_player_df.competition_id=="GB1") | (app_comp_player_df.competition_id=="ES1") | 
                   (app_comp_player_df.competition_id=="IT1") | (app_comp_player_df.competition_id=="L1") |
                   (app_comp_player_df.competition_id=="FR1") | (app_comp_player_df.competition_id=="PO1"))
#     app_comp_player_df = app_comp_player_df[cols2][league_cond].reset_index(drop=True)
    app_comp_player_df = app_comp_player_df[league_cond].reset_index(drop=True)

    # last_season 제대로 반영되지 않은 것들 발견하여, 새로 변경해준다
    # 6월을 넘지 않으면 직전 시즌 연도로 변경
    app_comp_player_df["last_season"] = app_comp_player_df["date"].apply(lambda x : x.year if x.month>=6 else x.year-1)

    # # 수치형 변수 column 리스트
    # group_cols = ["player_name", "last_season", "yellow_cards", "red_cards","goals", "assists", "minutes_played"]
    
    # # groupby를 통해, 이름, 연도 기준으로 수치형 변수 합산 -> 시즌별 기록 파악하기 위함
    # df_grouped = app_comp_player_df[group_cols].groupby(["player_name", "last_season"]).sum()
    # df_grouped.reset_index(inplace=True)

#     # merge 하기 위한 기준인 "player_name", "last_season"은 남겨두고 group_cols와 일치하지 않는 컬럼 삭재
#     df_not_grouped = app_comp_player_df.loc[:, ~app_comp_player_df.columns.isin(group_cols[2:])]

#     # 위에서 만든 df_grouped, df_not_grouped merge
#     df_merged = pd.merge(df_grouped, df_not_grouped, how="left", on=["player_name", "last_season"])
    
    return app_comp_player_df
    # return df_grouped
#     return df_merged

In [175]:
players_df.columns

Index(['player_id', 'first_name', 'last_name', 'name', 'last_season',
       'current_club_id', 'player_code', 'country_of_birth', 'city_of_birth',
       'country_of_citizenship', 'date_of_birth', 'sub_position', 'position',
       'foot', 'height_in_cm', 'contract_expiration_date', 'agent_name',
       'image_url', 'url', 'current_club_domestic_competition_id',
       'current_club_name', 'market_value_in_eur',
       'highest_market_value_in_eur'],
      dtype='object')

In [176]:
df_merged

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played
0,Aaron Boupendza,2019,0,0,0,0,90
1,Aaron Connolly,2019,0,0,2,1,878
2,Aaron Connolly,2020,0,0,3,2,1219
3,Aaron Connolly,2021,0,0,0,0,243
4,Aaron Connolly,2022,0,0,0,0,29
...,...,...,...,...,...,...,...
16615,İlkay Gündoğan,2019,7,0,3,2,1770
16616,İlkay Gündoğan,2020,1,0,14,4,2810
16617,İlkay Gündoğan,2021,1,0,10,4,2273
16618,İlkay Gündoğan,2022,1,0,8,5,2769


In [177]:
df_prac = app_comp_player_merge2(apps_copy, competitions_df, players_df)
df_prac

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,yellow_cards,red_cards,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url_y,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,3194817_105470,3194817,105470,31,31,2019-08-09,Alisson,GB1,0,0,...,right,193.0,2027-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/alisson/profil...,GB1,Liverpool Football Club,32000000.0,90000000.0
1,3194817_109751,3194817,109751,1123,1123,2019-08-09,Onel Hernández,GB1,0,0,...,right,172.0,2025-06-30 00:00:00,G1,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/onel-hernandez...,GB1,Norwich City,1000000.0,4000000.0
2,3194817_121385,3194817,121385,1123,1123,2019-08-09,Grant Hanley,GB1,0,0,...,right,188.0,2025-06-30 00:00:00,PLG,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/grant-hanley/p...,GB1,Norwich City,1500000.0,4500000.0
3,3194817_126600,3194817,126600,1123,1123,2019-08-09,Kenny McLean,GB1,0,0,...,left,180.0,2025-06-30 00:00:00,Trueno Sports,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/kenny-mclean/p...,GB1,Norwich City,1200000.0,4500000.0
4,3194817_131789,3194817,131789,31,31,2019-08-09,Roberto Firmino,GB1,0,0,...,right,181.0,2026-06-30 00:00:00,ROGON,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/roberto-firmin...,GB1,Liverpool Football Club,15000000.0,90000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290901,4113028_742521,4113028,742521,12321,12321,2024-02-26,Jhon Solís,ES1,0,0,...,right,186.0,2028-06-30 00:00:00,Jose Levis Manager,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/jhon-solis/pro...,ES1,Girona Fútbol Club S. A. D.,3000000.0,3000000.0
290902,4113028_743591,4113028,743591,12321,12321,2024-02-26,Sávio,ES1,1,0,...,left,176.0,2024-06-30 00:00:00,Promanager,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/savio/profil/s...,ES1,Girona Fútbol Club S. A. D.,30000000.0,30000000.0
290903,4113028_82503,4113028,82503,367,367,2024-02-26,Aridane Hernández,ES1,0,0,...,right,188.0,2025-06-30 00:00:00,JV SPORTS,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aridane-hernan...,ES1,Rayo Vallecano de Madrid S.A.D.,900000.0,4000000.0
290904,4113028_85295,4113028,85295,367,367,2024-02-26,Iván Balliu,ES1,0,0,...,right,172.0,2025-06-30 00:00:00,Fabián Bustos,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ivan-balliu/pr...,ES1,Rayo Vallecano de Madrid S.A.D.,3500000.0,4000000.0


In [178]:
group_cols = ["player_name", "last_season", "yellow_cards", "red_cards","goals", "assists", "minutes_played"]

# merge 하기 위한 기준인 "player_name", "last_season"은 남겨두고 group_cols와 일치하지 않는 컬럼 삭재
# df_not_grouped = df_prac.loc[:, ~df_prac.columns.isin(group_cols[2:])]
not_merged_cols = group_cols[2:] + ["appearance_id", "game_id", "date", "url_x", "image_url", "url_y"]
# df_not_grouped = df_prac.loc[:, ~df_prac.columns.isin(not_merged_cols)]
df_not_grouped = df_prac.loc[:, ~df_prac.columns.isin(not_merged_cols)].drop_duplicates()


# 위에서 만든 df_grouped, df_not_grouped merge
df_prac2 = pd.merge(df_merged, df_not_grouped, how="left", on=["player_name", "last_season"])
# not_merged_cols

In [179]:
df_prac2[:30]

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_club_id,player_current_club_id,...,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,Aaron Boupendza,2019,0,0,0,0,90,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Connolly,2019,0,0,2,1,878,434207.0,1237.0,1237.0,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
2,Aaron Connolly,2020,0,0,3,2,1219,434207.0,1237.0,1237.0,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
3,Aaron Connolly,2021,0,0,0,0,243,434207.0,1237.0,1237.0,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
4,Aaron Connolly,2022,0,0,0,0,29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Aaron Cresswell,2019,7,0,3,0,2115,92571.0,379.0,379.0,...,Left-Back,Defender,left,NaN,2024-06-30 00:00:00,Unique Sports Group,GB1,West Ham United Football Club,900000.0,12000000.0
6,Aaron Cresswell,2020,3,0,0,8,4014,92571.0,379.0,379.0,...,Left-Back,Defender,left,NaN,2024-06-30 00:00:00,Unique Sports Group,GB1,West Ham United Football Club,900000.0,12000000.0
7,Aaron Cresswell,2021,3,0,2,4,2785,92571.0,379.0,379.0,...,Left-Back,Defender,left,NaN,2024-06-30 00:00:00,Unique Sports Group,GB1,West Ham United Football Club,900000.0,12000000.0
8,Aaron Cresswell,2022,2,0,0,1,2250,92571.0,379.0,379.0,...,Left-Back,Defender,left,NaN,2024-06-30 00:00:00,Unique Sports Group,GB1,West Ham United Football Club,900000.0,12000000.0
9,Aaron Cresswell,2023,2,0,0,0,225,92571.0,379.0,379.0,...,Left-Back,Defender,left,NaN,2024-06-30 00:00:00,Unique Sports Group,GB1,West Ham United Football Club,900000.0,12000000.0


In [160]:
len(df_prac2.columns)

38

In [180]:
df_prac2

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_club_id,player_current_club_id,...,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,Aaron Boupendza,2019,0,0,0,0,90,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Aaron Connolly,2019,0,0,2,1,878,434207.0,1237.0,1237.0,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
2,Aaron Connolly,2020,0,0,3,2,1219,434207.0,1237.0,1237.0,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
3,Aaron Connolly,2021,0,0,0,0,243,434207.0,1237.0,1237.0,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
4,Aaron Connolly,2022,0,0,0,0,29,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17969,İlkay Gündoğan,2019,7,0,3,2,1770,53622.0,281.0,131.0,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0
17970,İlkay Gündoğan,2020,1,0,14,4,2810,53622.0,281.0,131.0,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0
17971,İlkay Gündoğan,2021,1,0,10,4,2273,53622.0,281.0,131.0,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0
17972,İlkay Gündoğan,2022,1,0,8,5,2769,53622.0,281.0,131.0,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0


In [163]:
df_prac2.iloc[:, :10]

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_club_id,player_current_club_id
0,Aaron Boupendza,2019,0,0,0,0,90,427605,3349,7775
1,Aaron Connolly,2019,0,0,2,1,878,434207,1237,1237
2,Aaron Connolly,2019,0,0,2,1,878,434207,1237,1237
3,Aaron Connolly,2020,0,0,3,2,1219,434207,1237,1237
4,Aaron Connolly,2021,0,0,0,0,243,434207,1237,1237
...,...,...,...,...,...,...,...,...,...,...
29442,İlkay Gündoğan,2022,1,0,8,5,2769,53622,281,131
29443,İlkay Gündoğan,2023,3,0,7,7,2538,53622,281,131
29444,İlkay Gündoğan,2023,3,0,7,7,2538,53622,131,131
29445,İlkay Gündoğan,2023,3,0,7,7,2538,53622,131,131


In [165]:
df_prac2.iloc[:, :20]

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played,player_id,player_club_id,player_current_club_id,competition_id,competition_code,name_x,sub_type,type,country_id,country_name,domestic_league_code,confederation,first_name
0,Aaron Boupendza,2019,0,0,0,0,90,427605,3349,7775,POCP,allianz-cup,allianz-cup,domestic_cup,domestic_cup,136,Portugal,PO1,europa,Aaron
1,Aaron Connolly,2019,0,0,2,1,878,434207,1237,1237,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,Aaron
2,Aaron Connolly,2019,0,0,2,1,878,434207,1237,1237,FAC,fa-cup,fa-cup,domestic_cup,domestic_cup,189,England,GB1,europa,Aaron
3,Aaron Connolly,2020,0,0,3,2,1219,434207,1237,1237,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,Aaron
4,Aaron Connolly,2021,0,0,0,0,243,434207,1237,1237,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,Aaron
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29442,İlkay Gündoğan,2022,1,0,8,5,2769,53622,281,131,FAC,fa-cup,fa-cup,domestic_cup,domestic_cup,189,England,GB1,europa,İlkay
29443,İlkay Gündoğan,2023,3,0,7,7,2538,53622,281,131,FAC,fa-cup,fa-cup,domestic_cup,domestic_cup,189,England,GB1,europa,İlkay
29444,İlkay Gündoğan,2023,3,0,7,7,2538,53622,131,131,ES1,laliga,laliga,first_tier,domestic_league,157,Spain,ES1,europa,İlkay
29445,İlkay Gündoğan,2023,3,0,7,7,2538,53622,131,131,CDR,copa-del-rey,copa-del-rey,domestic_cup,domestic_cup,157,Spain,ES1,europa,İlkay


In [121]:
df_not_grouped.shape

(315277, 37)

In [120]:
df_not_grouped.loc[:, ~df_not_grouped.columns.isin(["url_x", "image_url", "url_y"])].drop_duplicates().sort_values("player_name")

,appearance_id,player_id,player_club_id,player_current_club_id,player_name,competition_id,competition_code,name_x,sub_type,type,...,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
309,3229023_427605,427605,3349,7775,Aaron Boupendza,POCP,allianz-cup,allianz-cup,domestic_cup,domestic_cup,...,Centre-Forward,Attack,left,180.0,2025-06-30 00:00:00,VV Consulting,TR1,Hatayspor Futbol Kulübü,6500000.0,8500000.0
63838,3420826_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
51828,3219160_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
81346,3421774_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
65851,3421694_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228695,3838275_53622,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0
46547,3219135_53622,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0
42145,3219229_53622,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0
13166,3219401_53622,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,...,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,ES1,Futbol Club Barcelona,18000000.0,50000000.0


In [157]:
df_not_grouped[df_not_grouped["player_name"] == "Aaron Connolly"].iloc[:, :20]

,player_id,player_club_id,player_current_club_id,player_name,competition_id,competition_code,name_x,sub_type,type,country_id,country_name,domestic_league_code,confederation,first_name,last_name,name_y,last_season,current_club_id,player_code,country_of_birth
5321,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,Aaron,Connolly,Aaron Connolly,2019,1237,aaron-connolly,Ireland
30668,434207,1237,1237,Aaron Connolly,FAC,fa-cup,fa-cup,domestic_cup,domestic_cup,189,England,GB1,europa,Aaron,Connolly,Aaron Connolly,2019,1237,aaron-connolly,Ireland
50969,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,Aaron,Connolly,Aaron Connolly,2020,1237,aaron-connolly,Ireland
132795,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,Aaron,Connolly,Aaron Connolly,2021,1237,aaron-connolly,Ireland
169259,434207,641,1237,Aaron Connolly,FAC,fa-cup,fa-cup,domestic_cup,domestic_cup,189,England,GB1,europa,Aaron,Connolly,Aaron Connolly,2021,1237,aaron-connolly,Ireland
229336,434207,3008,1237,Aaron Connolly,FAC,fa-cup,fa-cup,domestic_cup,domestic_cup,189,England,GB1,europa,Aaron,Connolly,Aaron Connolly,2022,1237,aaron-connolly,Ireland


In [153]:
df_not_grouped.columns[14]

'first_name'

In [150]:
df_not_grouped[df_not_grouped["player_name"] == "Aaron Connolly"].iloc[:, 14:20].duplicated()


5321      False
30668      True
50969     False
132795    False
169259     True
229336    False
dtype: bool

In [126]:
df_not_grouped[df_not_grouped["player_name"] == "Aaron Connolly"].iloc[:, 20:].duplicated()
# df_not_grouped[df_not_grouped["player_name"] == "Aaron Connolly"].drop_duplicates()



5321      False
6902       True
8378       True
11045      True
12532      True
14083      True
15579      True
18005      True
20386      True
22708      True
24187      True
28761      True
29544      True
30256      True
30668      True
33794      True
34598      True
42778      True
50969      True
51828      True
53983      True
55193      True
56465      True
57325      True
61421      True
63838      True
65013      True
65851      True
68090      True
70391      True
74817      True
78470      True
81346      True
82512      True
85522      True
88617      True
98709      True
99960      True
100894     True
106579     True
120402     True
129348     True
132795     True
141840     True
155696     True
159067     True
169259     True
181316     True
229336     True
dtype: bool

In [107]:
df_not_grouped.sort_values(by="player_name")[:30]

,appearance_id,player_id,player_club_id,player_current_club_id,player_name,competition_id,competition_code,name_x,sub_type,type,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url_y,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
309,3229023_427605,427605,3349,7775,Aaron Boupendza,POCP,allianz-cup,allianz-cup,domestic_cup,domestic_cup,...,left,180.0,2025-06-30 00:00:00,VV Consulting,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-boupendz...,TR1,Hatayspor Futbol Kulübü,6500000.0,8500000.0
63838,3420826_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
51828,3219160_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
81346,3421774_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
65851,3421694_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
106579,3429752_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
11045,3219407_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
85522,3421803_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
57325,3219120_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
132795,3583521_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0


In [115]:
df_not_grouped.sort_values(by="player_name")[30:]

,appearance_id,player_id,player_club_id,player_current_club_id,player_name,competition_id,competition_code,name_x,sub_type,type,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url_y,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
24187,3219334_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
56465,3219206_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
159067,3592053_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
129348,3429863_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
53983,3219171_434207,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228695,3838275_53622,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
46547,3219135_53622,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
42145,3219229_53622,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
13166,3219401_53622,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,...,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0


In [105]:
df_merged

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played
0,Aaron Boupendza,2019,0,0,0,0,90
1,Aaron Connolly,2019,0,0,2,1,878
2,Aaron Connolly,2020,0,0,3,2,1219
3,Aaron Connolly,2021,0,0,0,0,243
4,Aaron Connolly,2022,0,0,0,0,29
...,...,...,...,...,...,...,...
16615,İlkay Gündoğan,2019,7,0,3,2,1770
16616,İlkay Gündoğan,2020,1,0,14,4,2810
16617,İlkay Gündoğan,2021,1,0,10,4,2273
16618,İlkay Gündoğan,2022,1,0,8,5,2769


In [101]:
df_prac2[df_prac2.columns[:20]]

,player_name,last_season,yellow_cards,red_cards,goals,assists,minutes_played,appearance_id,player_id,player_club_id,player_current_club_id,competition_id,competition_code,name_x,sub_type,type,country_id,country_name,domestic_league_code,confederation
0,Aaron Boupendza,2019,0,0,0,0,90,3229023_427605,427605,3349,7775,POCP,allianz-cup,allianz-cup,domestic_cup,domestic_cup,136,Portugal,PO1,europa
1,Aaron Connolly,2019,0,0,2,1,878,3219422_434207,434207,1237,1237,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa
2,Aaron Connolly,2019,0,0,2,1,878,3219418_434207,434207,1237,1237,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa
3,Aaron Connolly,2019,0,0,2,1,878,3219072_434207,434207,1237,1237,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa
4,Aaron Connolly,2019,0,0,2,1,878,3219407_434207,434207,1237,1237,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315272,İlkay Gündoğan,2023,3,0,7,7,2538,4112818_53622,53622,131,131,ES1,laliga,laliga,first_tier,domestic_league,157,Spain,ES1,europa
315273,İlkay Gündoğan,2023,3,0,7,7,2538,4113040_53622,53622,131,131,ES1,laliga,laliga,first_tier,domestic_league,157,Spain,ES1,europa
315274,İlkay Gündoğan,2023,3,0,7,7,2538,4112886_53622,53622,131,131,ES1,laliga,laliga,first_tier,domestic_league,157,Spain,ES1,europa
315275,İlkay Gündoğan,2023,3,0,7,7,2538,4113180_53622,53622,131,131,ES1,laliga,laliga,first_tier,domestic_league,157,Spain,ES1,europa


In [96]:
df_prac2[df_prac2.columns[20:30]]


,url_x,first_name,last_name,name_y,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth
0,https://www.transfermarkt.co.uk/allianz-cup/st...,Aaron,Boupendza,Aaron Boupendza,7775,aaron-boupendza,Gabon,Moanda,Gabon,1996-08-07
1,https://www.transfermarkt.co.uk/premier-league...,Aaron,Connolly,Aaron Connolly,1237,aaron-connolly,Ireland,Galway,Ireland,2000-01-28
2,https://www.transfermarkt.co.uk/premier-league...,Aaron,Connolly,Aaron Connolly,1237,aaron-connolly,Ireland,Galway,Ireland,2000-01-28
3,https://www.transfermarkt.co.uk/premier-league...,Aaron,Connolly,Aaron Connolly,1237,aaron-connolly,Ireland,Galway,Ireland,2000-01-28
4,https://www.transfermarkt.co.uk/premier-league...,Aaron,Connolly,Aaron Connolly,1237,aaron-connolly,Ireland,Galway,Ireland,2000-01-28
...,...,...,...,...,...,...,...,...,...,...
315272,https://www.transfermarkt.co.uk/laliga/startse...,İlkay,Gündoğan,İlkay Gündoğan,131,ilkay-gundogan,Germany,Gelsenkirchen,Germany,1990-10-24
315273,https://www.transfermarkt.co.uk/laliga/startse...,İlkay,Gündoğan,İlkay Gündoğan,131,ilkay-gundogan,Germany,Gelsenkirchen,Germany,1990-10-24
315274,https://www.transfermarkt.co.uk/laliga/startse...,İlkay,Gündoğan,İlkay Gündoğan,131,ilkay-gundogan,Germany,Gelsenkirchen,Germany,1990-10-24
315275,https://www.transfermarkt.co.uk/laliga/startse...,İlkay,Gündoğan,İlkay Gündoğan,131,ilkay-gundogan,Germany,Gelsenkirchen,Germany,1990-10-24


In [97]:
df_prac2[df_prac2.columns[30:]]


,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url_y,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,Centre-Forward,Attack,left,180.0,2025-06-30 00:00:00,VV Consulting,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-boupendz...,TR1,Hatayspor Futbol Kulübü,6500000.0,8500000.0
1,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
2,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
3,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
4,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...
315272,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
315273,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
315274,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
315275,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0


In [57]:
df_not_grouped

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,competition_code,name_x,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url_y,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
0,3213705_243779,3213705,243779,10140,2995,2019-07-27,Adriano Castanheira,POCP,allianz-cup,allianz-cup,...,left,178.0,2023-06-30 00:00:00,NCfoot,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/adriano-castan...,PO1,FC Paços de Ferreira,200000.0,600000.0
1,3182926_100986,3182926,100986,583,449,2019-08-03,Thomas Meunier,FRCH,trophee-des-champions,trophee-des-champions,...,right,191.0,2025-06-30 00:00:00,Thomas Meunier,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/thomas-meunier...,TR1,Trabzonspor Kulübü,2500000.0,35000000.0
2,3182926_102558,3182926,102558,583,583,2019-08-03,Marco Verratti,FRCH,trophee-des-champions,trophee-des-champions,...,right,165.0,2026-06-30 00:00:00,Rafaela Pimenta,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/marco-verratti...,FR1,Paris Saint-Germain Football Club,25000000.0,75000000.0
3,3182926_120629,3182926,120629,583,379,2019-08-03,Alphonse Areola,FRCH,trophee-des-champions,trophee-des-champions,...,right,195.0,2027-06-30 00:00:00,Rafaela Pimenta,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/alphonse-areol...,GB1,West Ham United Football Club,10000000.0,17500000.0
4,3182926_126719,3182926,126719,583,294,2019-08-03,Juan Bernat,FRCH,trophee-des-champions,trophee-des-champions,...,left,170.0,2024-06-30 00:00:00,InterStarDeporte,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/juan-bernat/pr...,PO1,Sport Lisboa e Benfica,6000000.0,20000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
315272,4113028_742521,4113028,742521,12321,12321,2024-02-26,Jhon Solís,ES1,laliga,laliga,...,right,186.0,2028-06-30 00:00:00,Jose Levis Manager,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/jhon-solis/pro...,ES1,Girona Fútbol Club S. A. D.,3000000.0,3000000.0
315273,4113028_743591,4113028,743591,12321,12321,2024-02-26,Sávio,ES1,laliga,laliga,...,left,176.0,2024-06-30 00:00:00,Promanager,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/savio/profil/s...,ES1,Girona Fútbol Club S. A. D.,30000000.0,30000000.0
315274,4113028_82503,4113028,82503,367,367,2024-02-26,Aridane Hernández,ES1,laliga,laliga,...,right,188.0,2025-06-30 00:00:00,JV SPORTS,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aridane-hernan...,ES1,Rayo Vallecano de Madrid S.A.D.,900000.0,4000000.0
315275,4113028_85295,4113028,85295,367,367,2024-02-26,Iván Balliu,ES1,laliga,laliga,...,right,172.0,2025-06-30 00:00:00,Fabián Bustos,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ivan-balliu/pr...,ES1,Rayo Vallecano de Madrid S.A.D.,3500000.0,4000000.0


In [62]:
df_not_grouped.sort_values(by="player_name")

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,date,player_name,competition_id,competition_code,name_x,...,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url_y,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
309,3229023_427605,3229023,427605,3349,7775,2019-08-05,Aaron Boupendza,POCP,allianz-cup,allianz-cup,...,left,180.0,2025-06-30 00:00:00,VV Consulting,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-boupendz...,TR1,Hatayspor Futbol Kulübü,6500000.0,8500000.0
63838,3420826_434207,3420826,434207,1237,1237,2020-09-14,Aaron Connolly,GB1,premier-league,premier-league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
51828,3219160_434207,3219160,434207,1237,1237,2020-06-23,Aaron Connolly,GB1,premier-league,premier-league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
81346,3421774_434207,3421774,434207,1237,1237,2020-12-07,Aaron Connolly,GB1,premier-league,premier-league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
65851,3421694_434207,3421694,434207,1237,1237,2020-09-26,Aaron Connolly,GB1,premier-league,premier-league,...,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228695,3838275_53622,3838275,53622,281,131,2023-01-05,İlkay Gündoğan,GB1,premier-league,premier-league,...,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
46547,3219135_53622,3219135,53622,281,131,2020-03-08,İlkay Gündoğan,GB1,premier-league,premier-league,...,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
42145,3219229_53622,3219229,53622,281,131,2020-02-19,İlkay Gündoğan,GB1,premier-league,premier-league,...,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
13166,3219401_53622,3219401,53622,281,131,2019-10-06,İlkay Gündoğan,GB1,premier-league,premier-league,...,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0


In [74]:
a = df_not_grouped.loc[:, ~df_not_grouped.columns.isin(["date"])].columns
len(a)

38

In [76]:
df_not_grouped.loc[:, ~df_not_grouped.columns.isin(["date"])].sort_values(by="player_name")[a[:20]]

,appearance_id,game_id,player_id,player_club_id,player_current_club_id,player_name,competition_id,competition_code,name_x,sub_type,type,country_id,country_name,domestic_league_code,confederation,url_x,first_name,last_name,name_y,last_season
309,3229023_427605,3229023,427605,3349,7775,Aaron Boupendza,POCP,allianz-cup,allianz-cup,domestic_cup,domestic_cup,136,Portugal,PO1,europa,https://www.transfermarkt.co.uk/allianz-cup/st...,Aaron,Boupendza,Aaron Boupendza,2019
63838,3420826_434207,3420826,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,https://www.transfermarkt.co.uk/premier-league...,Aaron,Connolly,Aaron Connolly,2020
51828,3219160_434207,3219160,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,https://www.transfermarkt.co.uk/premier-league...,Aaron,Connolly,Aaron Connolly,2020
81346,3421774_434207,3421774,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,https://www.transfermarkt.co.uk/premier-league...,Aaron,Connolly,Aaron Connolly,2020
65851,3421694_434207,3421694,434207,1237,1237,Aaron Connolly,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,https://www.transfermarkt.co.uk/premier-league...,Aaron,Connolly,Aaron Connolly,2020
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228695,3838275_53622,3838275,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,https://www.transfermarkt.co.uk/premier-league...,İlkay,Gündoğan,İlkay Gündoğan,2022
46547,3219135_53622,3219135,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,https://www.transfermarkt.co.uk/premier-league...,İlkay,Gündoğan,İlkay Gündoğan,2019
42145,3219229_53622,3219229,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,https://www.transfermarkt.co.uk/premier-league...,İlkay,Gündoğan,İlkay Gündoğan,2019
13166,3219401_53622,3219401,53622,281,131,İlkay Gündoğan,GB1,premier-league,premier-league,first_tier,domestic_league,189,England,GB1,europa,https://www.transfermarkt.co.uk/premier-league...,İlkay,Gündoğan,İlkay Gündoğan,2019


In [78]:
df_not_grouped.loc[:, ~df_not_grouped.columns.isin(["date"])].sort_values(by="player_name")[a[20:]]

,current_club_id,player_code,country_of_birth,city_of_birth,country_of_citizenship,date_of_birth,sub_position,position,foot,height_in_cm,contract_expiration_date,agent_name,image_url,url_y,current_club_domestic_competition_id,current_club_name,market_value_in_eur,highest_market_value_in_eur
309,7775,aaron-boupendza,Gabon,Moanda,Gabon,1996-08-07,Centre-Forward,Attack,left,180.0,2025-06-30 00:00:00,VV Consulting,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-boupendz...,TR1,Hatayspor Futbol Kulübü,6500000.0,8500000.0
63838,1237,aaron-connolly,Ireland,Galway,Ireland,2000-01-28,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
51828,1237,aaron-connolly,Ireland,Galway,Ireland,2000-01-28,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
81346,1237,aaron-connolly,Ireland,Galway,Ireland,2000-01-28,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
65851,1237,aaron-connolly,Ireland,Galway,Ireland,2000-01-28,Centre-Forward,Attack,right,174.0,2024-06-30 00:00:00,NaN,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/aaron-connolly...,GB1,Brighton and Hove Albion Football Club,3500000.0,7000000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228695,131,ilkay-gundogan,Germany,Gelsenkirchen,Germany,1990-10-24,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
46547,131,ilkay-gundogan,Germany,Gelsenkirchen,Germany,1990-10-24,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
42145,131,ilkay-gundogan,Germany,Gelsenkirchen,Germany,1990-10-24,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
13166,131,ilkay-gundogan,Germany,Gelsenkirchen,Germany,1990-10-24,Central Midfield,Midfield,right,180.0,2025-06-30 00:00:00,KIN Partners,https://img.a.transfermarkt.technology/portrai...,https://www.transfermarkt.co.uk/ilkay-gundogan...,ES1,Futbol Club Barcelona,18000000.0,50000000.0
